In [4]:
import sys

import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow.keras
import tensorflow as tf
import matplotlib.pyplot as plt

# See versions
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Kerkas Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Scikit-Learn {sk.__version__}")

# Check for TensorFlow GPU access
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is ", "available" if gpu else "NOT AVAIALBLE")
print(tf.config.list_physical_devices())

Tensor Flow Version: 2.8.0
Kerkas Version: 2.8.0

Python 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]
Scikit-Learn 1.0.2
GPU is  available
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 1s 0us/step


In [23]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [11]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.3432305 ,  0.44998842, -0.2050588 , -0.14185855, -0.5354118 ,
         0.3411888 , -0.16038626, -0.21683082, -0.2064195 , -1.4316244 ]],
      dtype=float32)

In [12]:
tf.nn.softmax(predictions).numpy()
# It is possible to bake the tf.nn.softmax function into the activation function for the last layer of the network.
# While this can make the model output more directly interpretable, 
# this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation
# for all models when using a softmax output.

array([[0.15056325, 0.16752645, 0.08701617, 0.09269312, 0.06253591,
        0.15025616, 0.09099153, 0.08599783, 0.08689785, 0.02552168]],
      dtype=float32)

In [13]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [14]:
loss_fn(y_train[:1], predictions).numpy()

1.8954135

In [15]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [18]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0374 - accuracy: 0.9875
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0327 - accuracy: 0.9891
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0303 - accuracy: 0.9901
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0299 - accuracy: 0.9901
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0283 - accuracy: 0.9901


In [20]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0717 - accuracy: 0.9809 - 792ms/epoch - 3ms/step


[0.0716969296336174, 0.9809000492095947]

In [21]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [22]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.3507220e-09, 9.3451157e-11, 2.6196141e-09, 2.2272539e-05,
        6.5496775e-14, 5.0740288e-09, 1.1955998e-18, 9.9996901e-01,
        6.8232535e-09, 8.6482660e-06],
       [2.6750434e-17, 7.7349988e-08, 9.9999988e-01, 5.2454621e-11,
        7.3881126e-26, 4.3106146e-13, 1.7073465e-14, 4.0051584e-26,
        3.9312997e-09, 1.1389389e-25],
       [1.8322761e-12, 9.9997485e-01, 7.2841353e-06, 2.1803155e-08,
        2.3821721e-07, 2.4712339e-09, 1.3295383e-07, 4.3028871e-07,
        1.7015705e-05, 9.9590378e-11],
       [9.9998701e-01, 4.8689631e-15, 1.3056052e-07, 5.7864625e-12,
        1.0140113e-08, 4.1623513e-10, 1.0673426e-05, 1.5461067e-07,
        8.2109597e-11, 2.0435682e-06],
       [2.2993037e-08, 9.7150586e-13, 6.5498007e-08, 2.3505285e-11,
        9.9626547e-01, 1.7361520e-11, 9.9751263e-10, 6.0233011e-05,
        7.4976345e-11, 3.6742166e-03]], dtype=float32)>